# Estimating the Quantum Speed Limit of an Entangling Two-Qubit Gate

<font size="5">
This notebook demonstrates the estimation of the quantum speed limit of a unitary gate through the utilization of quantum optimal control. In this context, the quantum speed limit is defined as the shortest duration in which a unitary gate can be executed on a qubit array. The estimation is made possible through the application of Krotov's method of optimization in a minimal time approach.  

In [1]:
import os

import copy
from pathlib import Path

import numpy as np
import scipy

import qutip
from qutip import tensor,qeye,ket
from qutip.qip.circuit import t_gate
from qutip.qip.operations import berkeley,swap

import krotov
#import qdyn.model
#import qdyn.pulse
import subprocess as subp
import matplotlib.pyplot as plt

from multiprocessing import Pool

%matplotlib widget

/Users/davidanthonypohl/opt/anaconda3/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


<font size="5"> The objective is to implement a quantum gate at the quantum speed limit, which is the shortest duration that is theoretically permitted, on a specific three-qubit system. The system under consideration is a three-qubit chain that exhibits variation in qubit-qubit couplings and interaction strengths. As a consequence of these specific non-uniform interactions between the qubits, the system is referred to as an asymmetric chain and is characterised by the following Hamiltonian:
\begin{equation}
\hat{H}_0 = \sum_{i = 1}^{3} \frac{\omega_{i}}{2} \hat{\sigma}_z^{(i)} + \kappa_{x,x}^{(1,2)} \hat{\sigma}_x^{(1)}\hat{\sigma}_x^{(2)} + \kappa_{z,z}^{(1,2)} \hat{\sigma}_z^{(1)}\hat{\sigma}_z^{(2)} + \kappa_{x,x}^{(2,3)} \hat{\sigma}_x^{(2)}\hat{\sigma}_x^{(3)}. 
\end{equation}
The asymmetric chain is imposed with two time-dependent local controls, described by
$$\hat{H}(t) = u_1(t)\hat{\sigma}_x^{(1)} + u_2(t)\hat{\sigma}_x^{(2)} .$$
For instance, $u_2(t)$ could relate to a time-dependent electric field and $\hat{\sigma}_x^{(2)}$ mediates how this electric field acts on the second qubit. Now, the task of implementing a unitary gate on the total system described by described by $\hat{H}_0 + \hat{H}(t)$, refers to the process of steering the system's unitary evolution to the desired target gate at final time.
The objective of this example, is to entangle the first- and the third qubit of the asymmetric chain using the B gate, which is defined as:
\begin{equation}
B = \left(\begin{matrix} \cos(\frac{\pi}{8}) & 0 & 0 & i\sin(\frac{\pi}{8})
\\0 & \cos(\frac{3\pi}{8}) & i\sin(\frac{3\pi}{8}) & 0\\ 0 & i\sin(\frac{3\pi}{8}) & \cos(\frac{3\pi}{8}) & 0 \\ i\sin(\frac{\pi}{8}) & 0 & 0 & \cos(\frac{\pi}{8}) \end{matrix}\right).
\end{equation}
As a side note, the B gate is the most efficient two-qubit gate in setups that involve superconducting qubits and interactions of the type $\hat{\sigma}_x^{(i)}\hat{\sigma}_x^{(j)}, \hat{\sigma}_y^{(i)}\hat{\sigma}_y^{(j)}$ and $\hat{\sigma}_z^{(i)}\hat{\sigma}_z^{(j)}$, see <font color = blue> https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.93.020502 </font> .
A series of SWAP gates defined as 
\begin{equation}
SWAP = \left(\begin{matrix} 1 & 0 & 0 & 0 \\0 & 0 & 1 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{matrix}\right) 
\end{equation}
is utilized to generate the desired form of entanglement between the first- and the third qubit:
\begin{equation}
B^{(1,3)} = (SWAP \otimes 1) (1 \otimes B) (SWAP \otimes 1)
\end{equation}
Finally, $B^{(1,3)}$ is the desired target gate.

In [8]:
# Define target gate

# B gate
B = berkeley(2)

# Identity gate
E = qeye(2)

# SWAP gate
S = swap(2)

# Target gate
B_13 = tensor(S,E)*tensor(E,B)*tensor(S,E)
Target = B_13
Target = np.array(Target)
target_gate = qutip.Qobj(Target)

<font size="5"> 
Although, the quantum speed limit of $B^{(1,3)}$ is a priori unkown, we have to start the optimization protocol at a duration, where we think that it is plausible that the gate can be implemented. It recommends to start the protocol with a large final time, however if data regarding gate durations and physical understanding of the relevant timescales is available, one could start with a shorter final time. Here, the protocol begins with final time $τ = 2$, which is based on available data regarding gate durations, physical understanding and reasoning.

To ensure, that the results illustrate true physical behaivour we have to set certain parameters in a specific manner. For instance, the numerical implementation of a time axis requires a discretization. Furthermore, in order to keep the discretizazion error low, we must ensure to utilize a sufficient amount of time points for a good approximation of the time axis.
For instance, one could require to accurately represent the highest pulse frequency. One "rule by thumb" is to describe the corresponding oscillation period by at least 10 time points.
\begin{equation}
\Delta E \Delta t \equiv \omega_{max} \Delta t \geq \frac{1}{2} \Longrightarrow \Delta t = \frac{1}{2 \omega_{max}}.
\end{equation}
Here, since $ \omega_{max} =  \omega_{2} = 2 \pi \nu_{2} $, it holds that
\begin{equation}
\Delta t = \frac{1}{2 \pi \nu_{2}} \approx 0.0018.
\end{equation}


In [5]:
# Final time - duration of the protocol, that is, the duration of the gate implementation
τ = 2

# delta t 
Δt = 0.0018

# Number of time points
n = int(τ/Δt + 1)

# Ramping time: At time t=0, the pulses take on the value 0, after the ramping time tr, the pulses approach higher amplitudes.
tr = (τ/10)

#lambda: inverse step width required for the (gradient-based) Krotov optimization.
λ = 0.008

# Furthermore, we initialize the list that will be passed to the optimizer later in this code.
L = [τ,tr,Δt,λ,target_gate]

In [5]:
# Here, the Hamiltonian of the asymmetric chain imposed with the two local controls is numerically implemented.

def get_Ham(ν1=5.0, ν2 = 5.3, ν3 = 5.5, T = τ, t_rise = tr):
    """
    time dependent control fields u1 and u2
    e.g. u1 is a product of a flattop blackman shape with ramping time t_rise and
    a cosine function that oscillates at the resonance frequency of the first qubit 
    """
    u1 = lambda t, args: krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(6/10*np.cos(ν1*t*2*np.pi))
    u2 = lambda t, args: krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(6/10*np.cos(ν2*t*2*np.pi))
    
    # Pauli matrices
    Z = qutip.operators.sigmaz()
    X = qutip.operators.sigmax()
    E = qeye(2)

    # qubit-qubit interactions and their interaction strengths e.g. X1X2 and k1
    X1X2 = tensor(X,X,E)
    X2X3 = tensor(E,X,X)
    Z1Z2 = tensor(Z,Z,E)
    
    k1 = 0.11
    k2 = 0.22
    k3 = 0.31

    """
    While drift describes the three uncoupled qubits, 
    coupl describes the qubit-qubit interactions in the asymmetric chain
    The total time independent Hamiltonian H0, is the sum of both terms:
    """
    drift = -1/2 * ( ν1*tensor(Z,E,E) + ν2*tensor(E,Z,E) + ν3*tensor(E,E,Z) ) 
    coupl = k1*X1X2 + k2*Z1Z2 + k3*X2X3
    
    H0 = drift + coupl
    Ham_0 = qutip.Qobj(np.array(H0))
    # Numerically, it is important to transform H0 into a numpy array and subsequently to a qutip. quantum object,
    # such that the optimizer recognizes Ham_0 as a 16x16 matrix.

    """
    Define local control operators seperately.
    The output is a nested list, which will be passed to the optimizer.
    """

    H1 = tensor(X,E,E)
    H1 = np.array(H1)
    Ham_1 = qutip.Qobj(H1)
    
    H2 = tensor(E,X,E)
    H2 = np.array(H2)
    Ham_2 = qutip.Qobj(H2)

    
    return [Ham_0, [Ham_1, np.vectorize(u1)], [Ham_2, np.vectorize(u2)] ]


In [9]:
# Define Model

def qdyn_model(
    Ham,
    T, # final_time
    nt, # number of time steps
    basis, # logical basis
    gate, # target gate
    runfolder,
    prop_method="exact",
    J_T_conv=1e-4,
    Δ_J_T_conv = 1e-7,
    iter_stop=200000,
    **pulse_oct_kwargs, # kwargs = keyword arguments
):
    """ Create qdyn-model
    """
    runfolder = Path(runfolder) # path to runfolder
    ! mkdir $runfolder # bash command 
    tgrid = qdyn.pulse.pulse_tgrid(T=T, nt=nt, t0=0)
    # For technical reasons, QDYN stores the pulses on the intervals of the time grid.
    # That means, the time grid for the pulse needs to be shifted
    #by dt/2 with respect to the time grid used for the propagation
    
    model = qdyn.model.LevelModel()
    # 
    for H in Ham:
        if isinstance(H, list):
            model.add_ham(
                H[0],
                pulse=qdyn.pulse.Pulse(
                    tgrid,
                    amplitude=H[1](tgrid, None),
                    time_unit="ns",
                    ampl_unit="GHz",
                    config_attribs={
                        **pulse_oct_kwargs
                    }, # Additional config data, for when generating a QDYN config
                    # file section describing the pulse (e.g. {'oct_shape': 'flattop', 't_rise': '10_ns'})
                ),
                op_unit="iu",
            )
        else:
            model.add_ham(H, op_unit="GHz")
            
    qdyn.io.write_cmplx_array(
        qutip.Qobj(gate).full().flatten(order="F"), str(runfolder/"gate.dat")
    )
    model.set_propagation(
        T, nt, time_unit="ns", prop_method=prop_method
    )
    model.set_oct("krotovpk", J_T_conv, 5000, iter_stop=iter_stop, continue_=False, delta_J_T_conv=Δ_J_T_conv)
    
    # Observables
    Z = qutip.operators.sigmaz()
    E = qeye(2)
    
    Z1 = qutip.Qobj(np.array(tensor(Z,E,E)))
    Z2 = qutip.Qobj(np.array(tensor(E,Z,E)))
    Z3 = qutip.Qobj(np.array(tensor(E,E,Z)))

    """
    By using Z1 as observable it is possible to monitor the population transfer
    between the two levels of the first qubit.
    """
    

    model.add_observable(Z1, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "1" )
    model.add_observable(Z2, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "2" )
    model.add_observable(Z3, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "3" )
    
    
    
    for i, b in enumerate(basis):
        model.add_state(b, f"basis{i}")
        

    Path(runfolder).mkdir(exist_ok=True, parents=True)
    model.write_to_runfolder(str(runfolder))

<font size="5"> 
The following is a numerical implementation of Krotov’s method of optimization. Furthermore, Krotov's method is an iterative, monotonically convergent optimization algorithm using gradient information to achieve convergence. The advantage over other gradient-based techniques is the ensurance of monotonic convergence, which means after a sufficient amount of iterations, the optimizer will find a local minimum, see <font color = blue > https://www.scipost.org/10.21468/SciPostPhys.7.6.080?acad_field_slug=all </font> .

In [7]:
def krotov_routine(L):
    
    [τ,tr,Δt,λ,target_gate] = L

    pulse_oct_kwargs = dict(oct_lambda_a=λ, oct_shape = "flattop",t_rise=qdyn.units.UnitFloat(tr, "ns"), t_fall=qdyn.units.UnitFloat(tr, "ns") )
    rf_p = "rf"+str(τ)
    qdyn_model(get_Ham(T = τ, t_rise = tr), τ, n,[qutip.basis(8,i) for i in range(0,8)], target_gate, rf_p  , **pulse_oct_kwargs)

    env = {**os.environ, "OMP_NUM_THREADS": "1"}
    with open("rf"+str(τ)+"/the_file_you_want_to_save_the_output_to.txt", "w") as f:
        subp.run(
            [
                "qdyn_optimize",
                f"--gate={rf_p}/gate.dat",
                "--basis=basis0,basis1,basis2,basis3,basis4,basis5,basis6,basis7",
                "--J_T=J_T_sm",
                "--internal-units=GHz_units.txt",
                f"--write-optimized-gate={rf_p}/final_gate.dat",
                f"{rf_p}",
            ],
            
            
            stderr=f, stdout=f, env=env
        )
        subp.run(["pwd"], stderr=f, stdout=f)

<font size="5"> 
It was demonstrated that employing Krotov's method in a minimal time approach can recover the quantum speed limit, see <font color = blue > https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.240501 </font> .
<font size="5"> 
The minimal time approach goes as follows: 
Krotov's method is  used to implement \({B^{(1,3)}}\) at \(T\). The optimization is considered successful when the gate is implemented with the desired infidelity, meaning that the value of \(J_{T,SM}\) falls below a certain threshold, such as \(10^{-4}\). If the optimization is successful, the process is repeated with a shorter final time \(T_1\). After \(N-1\) successful optimizations, it may happen that the algorithm fails to converge in the \(N\)th optimization for \(t = T_N\), indicating that the desired infidelity cannot be achieved. If this occurs, it might be beneficial to use different initial guess fields and repeat the optimization. Eventually, a candidate for the quantum speed limit will be identified, meaning the desired gate can be implemented with the required infidelity for some \(T \geq T_{QSL}\), while for \(T < T_{QSL}\), the algorithm fails to converge. (See the notebook on infidelity-curves)

In [ ]:
# "do_final_times":
# For the final time a = τ , we reduce c times the parameter b from a. The output is an array that consists of final times: a, a-b, a-2b, ... a-(c-1)b.
# These are c final times in total, which will be passed to the subsequent function "do_optimization_input"

# "do_optimization_input":
# Here, for each of the c final times created in # "do_final_times" lists are created. Every list P consist of a final_time, t_rise, delta_t, lambda_0 and the target gate.
# In other words, the list P contains information necessary for performing a single optimization at certain final time.

# "do_krotov_parallel": 
#The aim is to parallelize all of the optimizations. Ideally,  c krotov optimizations will be performed simultaneously.


def do_final_times(a, b, c):
    """
    a = final time
    b = decreasing parameter
    c = how often shall b be reduced from a
    """
    return np.arange(a, 0, -b)[:c]


def do_optimization_input(L,b,c):
 
    a = L[0]
    P = [[m] + L[1:5] for m in do_final_times(a, b, c)]
    
    return P


def do_parallel_krotov(L,b,c):
    P = do_optimization_input(L,b,c)    
    with  Pool(len(P)) as p:
        results = p.map(krotov_routine,P)
    
               
# For instance, for do_parallel_krotov(L,1,2) and τ = 2. The final times 2,1 will be passed to the subsequent functions.
# Furthermore, the runfolders rf2 and rf1 will be created, 
# Any runfolder contains information regarding the system, its dynamics and the numerics of the optimization.

<font size="5"> 
After the optimizations, we seek to get insight regarding the physics of our results:

In [ ]:
# One can compare the guess pulse(s) with the optimized pulse(s) by plotting each of those.
# If the desired gate infidelity was achieved, small deviations between those pulses, indicate that the initial pulse was a good guess.

t, pr = np.loadtxt("rf2/pulse1.dat", unpack=True)
guess_pul = qdyn.pulse.Pulse(t, amplitude=pr, time_unit="ns", ampl_unit="GHz")
guess_pul.plot()

t, pr, pi = np.loadtxt("rf2/pulse1.oct.dat", unpack=True)
pul = qdyn.pulse.Pulse(t, amplitude=pr+1j*pi, time_unit="ns", ampl_unit="GHz")
pul.plot()

In [ ]:
# Furthermore, one can plot the observables against the time to gain insight for the dynamics during the gate implementation.

def propagate_obs():
    env = {**os.environ, "OMP_NUM_THREADS": "1"}
    with open("another_output_to.txt", "w") as f:
        subp.run(
            [
                "qdyn_prop_traj", f"--write-all-states=AllStates.dat",
                f"--use-oct-pulses",
                f"--time-unit=ns",
                f"--state-label=basis2",
                f"rf2"
            ],
            stderr=f, stdout=f, env=env
        )
        subp.run(["pwd"], stderr=f, stdout=f)


def plot_obs(obs, tlist, xlimit=None):
    fig, ax = plt.subplots()
    if callable(pulse):
        obs = np.array([obs(t, None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel('time (ns)')
    if xlimit is not None:
        ax.set_xlim(xlimit)
    plt.show(fig)
        

propagate_obs()
t, Z1, Z2, Z3 = np.loadtxt("rf2/observable.dat", unpack=True)
plot_obs(Z1,t)